In [1]:
import pymssql
import pandas as pd
import numpy as np
import os
import re
import joblib
import sys 
import DSI_Capstone_Steemit
# from DSI_Capstone_Steemit.utils.utils import(
#     load_data_and_description,
#     load_raw
# )

# load_data_and_description(data_type = 'tfidf')
data_directory = '../data/'
import seaborn as sns
import matplotlib.pylab as plt
%matplotlib inline

In [2]:
authors = pd.read_csv(os.path.join(data_directory,'accounts.csv'))

In [3]:
from sklearn.preprocessing import FunctionTransformer
transformer = FunctionTransformer(np.log1p)
reputation_scaled = transformer.fit_transform(authors['reputation'])[0]
reputation_scaled = reputation_scaled - min(reputation_scaled) + 0.5

authors['reputation scaled'] = reputation_scaled

/usr/local/lib/python2.7/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/usr/local/lib/python2.7/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


In [5]:
def load_raw_cleaned():
    posts_path = os.path.join(data_directory, 'posts_raw_cleaned',
                              'posts_raw_cleaned.csv')
    df_posts = pd.read_csv(posts_path)
    return df_posts

raw = load_raw_cleaned()

In [6]:
raw = raw[(raw['depth'] == 0)]

In [7]:
idx_popular_athors = raw['author'].isin(authors['name'].str.lower().values)

In [8]:
authors = authors.ix[0:5000,:]

In [9]:
connections_list = []
for i,name in enumerate(authors.ix[:,'name'].values):
    idx_name = raw['body mentions'].str.count(name) > 0
    connections = raw.ix[idx_name,'author'].value_counts().reset_index()
    connections.columns = ['target','weight']
    connections['source'] = name
    connections_list.append(connections)


In [10]:
# Node size can be the 

In [11]:
df_network_x = pd.concat(connections_list)

In [12]:
from sklearn.preprocessing import MinMaxScaler

In [13]:
mm = MinMaxScaler(feature_range=(0,15))

In [14]:
scaled_lookup = authors.set_index('name').to_dict()['reputation scaled']

In [15]:
df_network_x['node_size'] = df_network_x['source'].replace(scaled_lookup)

In [16]:
import seaborn as sns
%matplotlib inline

In [18]:
df_network_x.to_csv('../data/networkx/df_network.csv')